In [2]:
from PIL import Image
import numpy as np

Загрузите картинку с помощью команды open. Если необходимо, можете изменить размер картинки с помощью метода resize.

In [ ]:
img = Image.open('chicago.jpg')#.resize(())
img

Преобразуйте картинку из объекта класса PIL в Numpy массив.

In [3]:
x = np.array(img, dtype=np.float32)
x.dtype

dtype('float32')

In [4]:
x.shape

(1414, 2121, 3)

Видно, что исходная цветная картинка содержит три канала - RGB. Преобразуем её в одноканальную либо с помощью специальной функции rgb2gray либо обычным усреднением по трём каналам.

In [5]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [ ]:
Image.fromarray(np.asarray(rgb2gray(x), dtype=np.uint8))

![chicago](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTrA7av4wltFDizgLxZYD0ek7n1thuH7d9_izw8mjs-1U3MthgltOqIlVsgEIyPnmkII_A&usqp=CAU)

In [ ]:
Image.fromarray(np.asarray(x.mean(axis=2), dtype=np.uint8))

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTrA7av4wltFDizgLxZYD0ek7n1thuH7d9_izw8mjs-1U3MthgltOqIlVsgEIyPnmkII_A&usqp=CAU)

Дальше будем работать с одноканальной картинкой, т.е. двумерной матрицей.

In [8]:
#y = rgb2gray(x)
y = x.mean(axis=2)
y

array([[73.      , 75.      , 76.      , ..., 37.666668, 37.666668,
        37.666668],
       [73.      , 75.      , 76.      , ..., 38.666668, 38.666668,
        37.666668],
       [73.      , 75.      , 76.      , ..., 38.666668, 38.666668,
        37.666668],
       ...,
       [64.666664, 64.666664, 63.666668, ...,  9.666667,  9.666667,
         9.666667],
       [61.666668, 61.666668, 59.666668, ...,  9.666667,  9.666667,
         9.666667],
       [59.666668, 58.666668, 56.666668, ...,  9.666667,  9.666667,
         9.666667]], dtype=float32)

In [9]:
y.shape

(1414, 2121)

Сделаем SVD разложение

In [10]:
U, S, V = np.linalg.svd(y, full_matrices=False)

In [11]:
U.shape

(1414, 1414)

In [12]:
V.shape

(1414, 2121)

Проверим что разложение корректное, соберём его обратно, перемножив матрицы

PIL объект можно при необходимости сохранить в файл на диск

In [14]:
Image.fromarray(np.asarray(Y_full, dtype=np.uint8)).save(f'full_rank.png')

**Задание для самостоятельной работы:**  
1. Рассмотреть малоранговые разложения
2. Получить картинки для разного значения ранга
3. Сохранить несколько картинок на диск
4. Сделать выводы

**Подсказки:**  
Не забудьте при преобразовании Numpy массива в объект PIL преобразовывать тип элементов в np.uint8.  
Если число находится вне диапазона [0, 255], сделайте клиппирование - все числа больше 255 приравняйте числу 255, все отрицательные числа сделайте равными нулю.

In [3]:
def clip(A):
    A[A > 255] = 255
    A[A < 0] = 0
    return A

img = Image.open('saya.jpg')#.resize(())
x = np.array(img, dtype=np.float32)
red, green, blue = x[:,:,0], x[:,:,1], x[:,:,2]

Ur, Sr, Vr = np.linalg.svd(red, full_matrices=False) # SVD разложения для каждого канала
Ug, Sg, Vg = np.linalg.svd(green, full_matrices=False)
Ub, Sb, Vb = np.linalg.svd(blue, full_matrices=False)

matFactor = 10

redPoints = clip(Ur[:, :matFactor].dot(np.diag(Sr[:matFactor])).dot(Vr[:matFactor, :]))
greenPoints = clip(Ug[:, :matFactor].dot(np.diag(Sg[:matFactor])).dot(Vg[:matFactor, :]))
bluePoints = clip(Ub[:, :matFactor].dot(np.diag(Sb[:matFactor])).dot(Vb[:matFactor, :]))

Points = np.concatenate((redPoints[:,:, np.newaxis], greenPoints[:,:, np.newaxis], bluePoints[:,:, np.newaxis]), axis=2)
Image.fromarray(np.asarray(Points, dtype=np.uint8)).save(f'{matFactor} шакалов.png')